In [23]:
import pandas as pd
import geopandas as gpd
from pprint import pprint
import matplotlib.pyplot as plt
import contextily as cx
import folium

import math
import copy

from geotext import GeoText

Read in the textfile that is to be georeferenced

In [12]:
textfile = "data/texts/aid_wiki.txt"

with open(textfile, "r") as wiki:
    lines = wiki.readlines()
    text = wiki.read().replace("\n", " ")
allWords = []
for line in lines:
    allWords.extend(line.strip().split(" "))

Match words in the textfile with placenames using geotext.

In [43]:
matches = GeoText(" ".join(lines))


Create Dictionary with the matched placenames as keys and dictionaries with the counts as values.

In [41]:
d = {}

for i, match in enumerate(matches.countries):
    if match in d.keys():
        d[match]["count"] += 1
    else:
        d[match] = {"count": 1}

In [ ]:
countrynames = pd.read_csv("data/geodict_github/countrynames.csv", names=["Short", "Long"])
countrypositions = pd.read_csv("data/geodict_github/countrypositions.csv", names=["Short", "Lat", "Lon"])

join = pd.merge(countrynames, countrypositions, on="Short")

Fill the dictionary with the coordinates of the placenames.

In [50]:
lenKeys = len(d)
logInterval = round(lenKeys/20, 0)

for i, (placename, attributes) in enumerate(d.items()):
    print(placename)
    tmp_df_values = join.query("Long == @placename").values
    print(tmp_df_values)
    break
    attributes["lat"] = tmp_df_values[indexOfLat]
    attributes["lon"] = tmp_df_values[indexOfLon]

    # give feedback to progress
    
    if (i % logInterval == 0 and i > 0) or i == lenKeys:
        print(f"{i} of {lenKeys} ({round((i/lenKeys)*100, 1)}%)", end='\r')

United States
[]


Create a geodataframe with the matched placenames

In [13]:
geo = pd.DataFrame(
    [ [place] + list(attributes.values()) for place, attributes in d.items()],
    columns=["name", "count", "lat", "lon"])


In [14]:


geo = gpd.GeoDataFrame(
    geo,
    geometry=gpd.points_from_xy(geo.lon, geo.lat),
    crs=4326
)

geojsonname = textfile[textfile.find("/")+1:textfile.find(".")][6:]
geo.to_file(f"data/geodataframes/{geojsonname}.geojson", driver='GeoJSON')

In [15]:
pointMap = gpd.GeoSeries.explore(
    geo,
    tiles="Stamen Watercolor",
    marker_type='circle',
    marker_kwds={'radius': 50000,
    'fill': True})

pointMap

In [16]:
from folium import plugins

# extract coordinate of geodataframe
coordinates = [[point.xy[1][0], point.xy[0][0]] for point in geo.geometry]

# create map
heatMap = folium.Map(location = [15,30], zoom_start = 2, tiles=None)

# add tiles
folium.TileLayer(tiles='Cartodb dark_matter', name="Dark").add_to(heatMap)
folium.TileLayer(tiles='stamen watercolor', name="Watercolor").add_to(heatMap)

# add points and markercluster
points = folium.FeatureGroup(name="Points", show=False).add_to(heatMap)
cluster = plugins.MarkerCluster(name="Cluster").add_to(heatMap)
for coordinate in coordinates:
    folium.Circle(coordinate).add_to(cluster)
    folium.Circle(coordinate).add_to(points)

# add heatmap
plugins.HeatMap(
    name = 'HeatMap',
    data = coordinates,
    min_opacity = 0.3
    ).add_to(heatMap)

# add layercontrol
folium.LayerControl(collapsed=False).add_to(heatMap)


heatMap